Import libraries.

In [73]:
import sympy as sp
import numpy as np

First, we define a symbolic variable $x$.

In [74]:
variable_x = sp.Symbol('x')
variable_x

x

Now, we try passing this variable into several SymPy functions to see if that works.

In [75]:
sp.sin(variable_x)

sin(x)

In [76]:
sp.exp(variable_x)

exp(x)

In [77]:
variable_x**2 + 5 * variable_x

x**2 + 5*x

Alright, I'm convinced that this will work.

Now, we just code up several functions that  "apply" the function to the variable.

## Constant Multiplication

In [78]:
def sympy_constant_function(
    sympy_variable_x: sp.Symbol, 
    parameter_A: float) -> float:

    try:
        return parameter_A * sympy_variable_x
    except:
        FLOAT_ZERO = 0.
        return FLOAT_ZERO

## Polynomials

In [79]:
def sympy_nth_degree_polynomial(
    sympy_variable_x: sp.Symbol, 
    list_of_polynomial_coefficients: list) -> float:

    try:
        result = 0
        for i, coefficient in enumerate(list_of_polynomial_coefficients):
            result += coefficient * (sympy_variable_x ** i)
        return result
    except:
        FLOAT_ZERO = 0.
        return FLOAT_ZERO

In [80]:
sympy_nth_degree_polynomial(variable_x, [1,2,3])

3*x**2 + 2*x + 1

## Logarithms:

In [81]:
def sympy_logarithmic_function(
    sympy_variable_x: sp.Symbol, 
    parameter_A: float,
    parameter_B: float) -> float:

    try:
        return parameter_A * sp.log(parameter_B * sympy_variable_x)
    except Exception as ERROR:
        print(f"Fuck")
        return 0.

In [82]:
sympy_logarithmic_function(variable_x, 4, 5.6)

4*log(5.6*x)

## Exponentials:

In [83]:
def sympy_exponential_function(
        sympy_variable_x: sp.Symbol,
        parameter_A: float,
        parameter_B: float) -> float:
    """
    # Description:
    --------------
    Calculate the exponential function A * e^(B * x).
    
    # Parameters:
    --------------
    :param x: The input value(s) where the function is evaluated (can be a numpy array).

    :param A: The scaling factor for the output.

    :param B: The scaling factor for the input.

    # Returns
    --------------
    :return: The result of the exponential function.
    """
    try:
        return parameter_A * sp.exp(parameter_B * sympy_variable_x)
    except Exception as ERROR:
        print(f"Fuck")
        return 0.

In [84]:
sympy_exponential_function(variable_x, 20, 99)

20*exp(99*x)

I think I got the hang of this. One more before we try `lambdify`.

## Sine:

In [85]:
def sympy_sine_function(
        sympy_variable_x: sp.Symbol,
        parameter_A: float,
        parameter_B: float,
        parameter_C: float) -> float:
    """
    # Description:
    --------------
    Calculate the sinusoidal function A * sin(B * x + C).
    
    # Parameters:
    --------------
    :param x: The input value(s) where the function is evaluated (can be a numpy array).

    :param A: The amplitude of the sine wave.

    :param B: The frequency scaling factor.

    :param C: The phase shift.

    # Returns
    --------------
    :return: The result of the sinusoidal function.
    """
    try:
        return parameter_A * sp.sin(parameter_B * sympy_variable_x + parameter_C)
    except Exception as ERROR:
        return 0.
    

In [86]:
sympy_sine_function(variable_x, 4, 20, -6)

4*sin(20*x - 6)

Actually, one more thing we need to do is code up the function generator using Sympy.

In [87]:
def sympy_generate_random_function(sympy_variable_x: sp.Symbol, depth: int) -> float:
    
    functions = [
        sympy_constant_function,
        sympy_exponential_function,
        sympy_logarithmic_function,
        sympy_sine_function
        ]

    result = sympy_variable_x
    
    for iteration in range(depth):
        function_index = np.random.randint(0, len(functions))
        function = functions[function_index]
        number_of_arguments_per_function = function.__code__.co_argcount
        function_parameters = np.round(np.random.uniform(0, 12, size = number_of_arguments_per_function - 2))
        result = function(result, 1, *function_parameters)

    return result

In [88]:
sympy_generate_random_function(variable_x, 2)

nan

That seems pretty easy. Let's just try it several times:

## Depth: 1

In [89]:
sympy_generate_random_function(variable_x, 1)

x

In [90]:
sympy_generate_random_function(variable_x, 1)

x

In [91]:
sympy_generate_random_function(variable_x, 1)

x

## Depth: 2

In [92]:
sympy_generate_random_function(variable_x, 2)

log(10.0*x)

In [93]:
sympy_generate_random_function(variable_x, 2)

log(9.0*log(2.0*x))

In [94]:
sympy_generate_random_function(variable_x, 2)

log(7.0*exp(7.0*x))

## Depth: 3

In [95]:
sympy_generate_random_function(variable_x, 3)

sin(1.0*sin(7.0*x + 4.0) + 10.0)

In [96]:
sympy_generate_random_function(variable_x, 3)

12.0*x**1.0

In [97]:
sympy_generate_random_function(variable_x, 3)

sin(4.0*exp(7.0*x) + 7.0)

## Depth: 5

In [98]:
sympy_generate_random_function(variable_x, 5)

sin(1.0*sin(7.0*x + 11.0) + 7.0)

## Depth: 7

In [99]:
sympy_generate_random_function(variable_x, 7)

exp(9.0*sin(6.0*sin(3.0*log(10.0*exp(12.0*sin(11.0*log(6.0*x) + 8.0))) + 9.0) + 7.0))

## Depth: 10

In [100]:
sympy_generate_random_function(variable_x, 10)

200859416110144.0*(sin(3.0*exp(8.0*x) + 5.0)**2.0)**6.0

Now, let's try Lambdify. We want to turn these symbiolic expressions into something we can plug in for $x$.

In [101]:
randomly_generated_function = sympy_generate_random_function(variable_x, 2)
numerical_expression = sp.lambdify(variable_x, randomly_generated_function, 'numpy')

In [102]:
randomly_generated_function

log(7.0*exp(8.0*x))

I should be able to evaluate this by passing in a number.

In [103]:
numerical_expression(5)

41.945910149055315

In [106]:
type(sp.log)

sympy.core.function.FunctionClass